In [363]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier

In [364]:
# creating a function that finds the best train test split size for a given model
def best_train_size(model, df, features):
    max_score = 0
    best_train = 0
    for x in np.linspace(0, 1, 21):
        if x == 0 or x == 1:
            continue
        x_train, x_test, y_train, y_test = train_test_split(features, df["Outcome"], train_size = x, random_state = 42)
        model.fit(x_train, y_train)
        score = model.score(x_test, y_test)
        if score > max_score:
            max_score = score
            best_train = x
    return max_score, best_train

In [365]:
# creating a fucntion that uses RFE to find the best combination of features for a given model
def feature_elim(model, feature_columns, x_train, x_test, y_train, y_test):
    best_score = 0
    best_x = 0
    for x in range(len(feature_columns)):
        rfe = RFE(estimator = model, n_features_to_select = x + 1)
        rfe.fit(x_train, y_train)
        score = rfe.score(x_test, y_test)
        if score > best_score:
            best_score = score
            best_x = x + 1
    final_rfe = RFE(estimator = model, n_features_to_select = best_x)
    final_rfe.fit(x_train, y_train)
    final_rfe.score(x_test, y_test)
    return best_score, best_x, final_rfe.support_

In [366]:
# loading in data
df = pd.read_csv("/Users/cartermain/Downloads/diabetes.csv")

In [367]:
# counting null values
print(df.isnull().sum())

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64


This will be a logistic regression model which will require feature standardization since all values are continuous but on different scales

In [368]:
# collecting features
features = df.drop(columns = "Outcome")

In [369]:
# standardizing features
scaler = StandardScaler()
scaler.fit(features)
scaler.transform(features)

array([[ 0.63994726,  0.84832379,  0.14964075, ...,  0.20401277,
         0.46849198,  1.4259954 ],
       [-0.84488505, -1.12339636, -0.16054575, ..., -0.68442195,
        -0.36506078, -0.19067191],
       [ 1.23388019,  1.94372388, -0.26394125, ..., -1.10325546,
         0.60439732, -0.10558415],
       ...,
       [ 0.3429808 ,  0.00330087,  0.14964075, ..., -0.73518964,
        -0.68519336, -0.27575966],
       [-0.84488505,  0.1597866 , -0.47073225, ..., -0.24020459,
        -0.37110101,  1.17073215],
       [-0.84488505, -0.8730192 ,  0.04624525, ..., -0.20212881,
        -0.47378505, -0.87137393]])

In [370]:
# train test splitting using random state to ensure every iteration uses the same subset
x_train, x_test, y_train, y_test = train_test_split(features, df["Outcome"], train_size = 0.7, random_state = 42)

In [371]:
# training and scoring the first iteration of the logistic regression model
lr = LogisticRegression(max_iter = 1000)
lr.fit(x_train, y_train)
print(lr.score(x_test, y_test))

0.7359307359307359


In [372]:
# checking absolute value of feature importances
importances = pd.DataFrame(columns = features.columns, data = abs(lr.coef_))
print(importances)

   Pregnancies   Glucose  BloodPressure  SkinThickness   Insulin       BMI  \
0     0.057768  0.035904        0.01087       0.001414  0.000984  0.109083   

   DiabetesPedigreeFunction       Age  
0                  0.374084  0.036011  


In [373]:
# running through function to determine best train size 
lr_highest_score, lr_best_train = best_train_size(model, df, features_2)
print(lr_highest_score, lr_best_train)

0.7864583333333334 0.5


In [375]:
# updating train test split with random state remaining as 42
x_train, x_test, y_train, y_test = train_test_split(features, df["Outcome"], train_size = lr_best_train, random_state = 42)

In [376]:
# refitting model with updated train split
lr.fit(x_train, y_train)
print(lr.score(x_test, y_test))

0.7864583333333334


Now we'll try to improve our model's performance via hyperparameter tuning

In [377]:
# running model through feature elimination function
lr_best_score, lr_best_x, lr_support = feature_elim(lr, features.columns, x_train, x_test, y_train, y_test)
print(lr_best_score, lr_best_x, lr_support)

0.7864583333333334 5 [ True  True False False False  True  True  True]


We were able to maximize model performance while dropping 3 features which will help us conserve computational space

In [378]:
# creating list of kept feature names to create a new feature set
lr_kept_features = []
for x in range(len(lr_support)):
    if lr_support[x] == True:
        lr_kept_features.append(features.columns[x])
    else:
        continue

In [380]:
# creating new, refined feature set
features_2 = df[lr_kept_features]

In [381]:
# scaling new feature set
scaler_2 = StandardScaler()
scaler_2.fit_transform(features_2)

array([[ 0.63994726,  0.84832379,  0.20401277,  0.46849198,  1.4259954 ],
       [-0.84488505, -1.12339636, -0.68442195, -0.36506078, -0.19067191],
       [ 1.23388019,  1.94372388, -1.10325546,  0.60439732, -0.10558415],
       ...,
       [ 0.3429808 ,  0.00330087, -0.73518964, -0.68519336, -0.27575966],
       [-0.84488505,  0.1597866 , -0.24020459, -0.37110101,  1.17073215],
       [-0.84488505, -0.8730192 , -0.20212881, -0.47378505, -0.87137393]])

In [382]:
# train test splitting refined feature set at best train size 
x_train_2, x_test_2, y_train_2, y_test_2 = train_test_split(features_2, df["Outcome"], train_size = lr_best_train, random_state = 42)

In [386]:
# updating baseline model with new feature set
lr.fit(x_train_2, y_train_2)
print(lr.score(x_test_2, y_test_2))

0.7864583333333334


Now that we have implemented feature elimination, let's take a look into hyperparameter tuning to make further improvements

In [384]:
# creating dictionary of hyperparameters to test within model
params = {"solver": ["newton-cg", "lbfgs", "liblinear", "sag", "saga"], "penalty": ["l1", "l2"], "C": [100, 10, 1.0, 0.1, 0.01]}

In [387]:
# using randomized search to find best hyperparameters with 75 iterations
random_search = RandomizedSearchCV(lr, params, n_iter = 75)
random_search.fit(x_train_2, y_train_2)

/Users/cartermain/opt/miniconda3/lib/python3.9/site-packages/sklearn/model_selection/_search.py:292: UserWarning: The total space of parameters 50 is smaller than n_iter=75. Running 50 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/Users/cartermain/opt/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/cartermain/opt/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/cartermain/opt/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/cartermain/opt/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the

/Users/cartermain/opt/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/cartermain/opt/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/cartermain/opt/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/cartermain/opt/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/cartermain/opt/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/cartermain/op

RandomizedSearchCV(estimator=LogisticRegression(max_iter=1000), n_iter=75,
                   param_distributions={'C': [100, 10, 1.0, 0.1, 0.01],
                                        'penalty': ['l1', 'l2'],
                                        'solver': ['newton-cg', 'lbfgs',
                                                   'liblinear', 'sag',
                                                   'saga']})

In [388]:
# concatenating and sorting to visualize performance with each set of hyperparameters in order of highest accuracy
best_params = pd.concat([pd.DataFrame(random_search.cv_results_["params"]), pd.DataFrame(random_search.cv_results_["mean_test_score"], columns=["Accuracy"])] ,axis=1)
print(best_params.sort_values("Accuracy", ascending = False).head())

       solver penalty      C  Accuracy
22  liblinear      l1   1.00  0.752768
17  liblinear      l2  10.00  0.752734
46      lbfgs      l2   0.01  0.750171
45  newton-cg      l2   0.01  0.750171
26      lbfgs      l2   1.00  0.750137


In [389]:
# finding best score with test values
print(random_search.score(x_test_2, y_test_2))

0.78125


Our best peformer within the random search grid was lower than our base model so we can roll forward with default hyperparameters

We were able to get relatively strong performance out of logistic regression, but let's see if we can find another classification model that will predict diabetes with even greater accuracy

In [421]:
# turning each feature into 0/1 classification based on if it is below or above the feature's mean
for x in features.columns:
    class_features[x + "_class"] = df[x].apply(lambda row: 0 if row <= df[x].mean() else 1)

In [422]:
# train test splitting with random state set to 42 in preparation for feature elimination and hyperparameter tuning
x_train_4, x_test_4, y_train_4, y_test_4 = train_test_split(class_features, df["Outcome"], train_size = 0.7, random_state = 42)

In [423]:
# running our first iteration of the random forest classification model
rf = RandomForestClassifier(n_estimators = 50)
rf.fit(x_train_4, y_train_4)
print(rf.score(x_test_4, y_test_4))

0.658008658008658


Not great, let's see if we can improve with some feature elimination and hyperparameter tuning

In [424]:
# making use of function to find best feature set
rf_highest_score, rf_num_features, rf_support = feature_elim(rf, class_features.columns, x_train_4, x_test_4, y_train_4, y_test_4)
print(rf_highest_score, rf_num_features, rf_support)

0.7142857142857143 3 [False  True False False  True False  True False]


In [425]:
print(rf_support)

[False  True False False  True False  True False]


In [426]:
# appending kept feature names to a list
kept_features = []
for x in range(len(class_features.columns)):
    if rf_support[x] == True:
        kept_features.append(class_features.columns[x])
    else: 
        continue

In [427]:
# creating revised features list
class_features_2 = class_features[kept_features]

In [428]:
# train test splitting revised feature set
x_train_5, x_test_5, y_train_5, y_test_5 = train_test_split(class_features_2, df["Outcome"], train_size = 0.7, random_state = 42)

In [429]:
# training in model with revised feature set
rf.fit(x_train_5, y_train_5)
print(rf.score(x_test_5, y_test_5))

0.7142857142857143


In [430]:
# using function to find best train test split
rf_max_score, rf_best_split = best_train_size(rf, df, class_features_2)
print(rf_max_score, rf_best_split)

0.7948717948717948 0.9500000000000001


In [431]:
# fitting revised feature set with best train size based on for loop results above
best_x_train, best_x_test, best_y_train, best_y_test = train_test_split(class_features_2, df["Outcome"], train_size = rf_best_split, random_state = 42)

In [432]:
# training model with newly-split data
rf.fit(best_x_train, best_y_train)
print(rf.score(best_x_test, best_y_test))

0.7948717948717948


Now we just need to run some hyperparameter tuning to ensure that we are getting the most out of our model with the default hyperparameters

In [433]:
# creating dictionary of hyperparameter set to test
rf_params = {"n_estimators": range(10, 300, 20), "criterion": ["gini", "entropy"], "max_depth": range(1, len(class_features_2.columns) + 1)}

In [434]:
# using random search again to find best hyperparameter set
randomsearch_2 = RandomizedSearchCV(rf, rf_params, n_iter = 55)
randomsearch_2.fit(best_x_train, best_y_train)

RandomizedSearchCV(estimator=RandomForestClassifier(n_estimators=50), n_iter=55,
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': range(1, 4),
                                        'n_estimators': range(10, 300, 20)})

In [435]:
# concatenating and sorting to visualize performance with each set of hyperparameters in order of highest accuracy
rf_best_params = pd.concat([pd.DataFrame(randomsearch_2.cv_results_["params"]), pd.DataFrame(randomsearch_2.cv_results_["mean_test_score"], columns=["Accuracy"])] ,axis=1)
print(rf_best_params.sort_values("Accuracy", ascending = False).head())

    n_estimators  max_depth criterion  Accuracy
41            10          2      gini  0.729712
42           110          2      gini  0.724280
34            90          2   entropy  0.722910
2             70          3      gini  0.720132
3             30          3      gini  0.720132


In [436]:
# finding best score and the hyperparameter set that got that score
print(randomsearch_2.best_estimator_)
print(randomsearch_2.score(best_x_test, best_y_test))

RandomForestClassifier(max_depth=2, n_estimators=10)
0.6410256410256411


Much like with the random search for logistic regression, we didn't find better performance outside of default values so this is about the best performance we'll see out of this and just a bit better than what we saw from our optimized logistic regression model. Moving forward, we'd use the optimized version of the random forest classifier to classify and predict probability of diabetes with ~79.5% accuracy